In [118]:
# !pip install protobuf==3.20.3
# !pip install --upgrade tokenizers
# !pip install crewai crewai_tools langchain_community python-dotenv
# !pip install exifread
# !pip install chardet

In [1]:
# from google.colab import userdata
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#use the below directories for local runtime
input_dir = "example-files/original-pics/"  # Replace with your input folder
downscaled_dir_for_web = "example-files/downscaled-pics/"  # Replace with your output folder
downscaled_dir_for_LLM = "example-files/super-downscaled-pics/"
selected_dir = "example-files/selected-pics/"  # Replace with your output folder
example_web_files = "example-files/example-markdown/"
watermark_filepath = "example-files/logo_watermark-youtube.png"
img_filter_threshold = 5.6

In [3]:
from PIL import Image
import imghdr
import exifread

def resize_images(input_folder, output_folder, min_dimension):
    """Resizes images from input_folder and saves them to output_folder with the minimum dimension specified.

    Args:
        input_folder: The folder containing the original images.
        output_folder: The folder to save the resized images.
        min_dimension: The minimum dimension (width or height) for the resized images.
    """
    os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        if os.path.isfile(image_path) and imghdr.what(image_path):
            try:
                with Image.open(image_path) as img:
                    width, height = img.size
                    if width < height:
                        new_width = min_dimension
                        new_height = int((min_dimension / width) * height)
                    else:
                        new_height = min_dimension
                        new_width = int((min_dimension / height) * width)

                    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                    resized_img.info['exif'] = img.info['exif']
                    output_path = os.path.join(output_folder, filename)
                    resized_img.save(output_path, exif=resized_img.info.get('exif'))  # Save with EXIF data
                    # print(f"Resized and saved {filename} to {output_folder}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

resize_images(input_dir, downscaled_dir_for_web, min_dimension=1080)
resize_images(input_dir, downscaled_dir_for_LLM, min_dimension=512)    #use lower res to save on LLM tokens

C:\Users\eirik\AppData\Local\Temp\ipykernel_106536\180193930.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [74]:
# import base64
import imghdr
import exifread
from pydantic import BaseModel, Field
from typing import List, Optional

class ImageObject(BaseModel):
    filename: str = Field(..., description="Filename of the image.")
    filepath_full: str = Field(..., description="Complete filepath for the image.")
    date_taken: Optional[str] = Field(None, description="Date the image was originally captured.")
    image_notes: Optional[str] = Field(None, description="Notes related to the image.")
    # base64_encoding: Optional[str] = Field(None, description="Base64 encoded string of the image file.")
    # base64 no longer needed with OAI vision tool
    title: Optional[str] = Field(None, description="Title of the image.")
    summary: Optional[str] = Field(None, description="Publication ready and web friendly summary of the image.")
    quality_score: Optional[float] = Field(None, ge=1, le=10, description="Quality score of the image based on assessment.")
    image_description: Optional[str] = Field(None, description="Objective description of what is in the image.")
    critical_assessment: Optional[str] = Field(None, description="Critical and qualitative assessment of the image.")
    

def images_to_objects(input_folder: str) -> List[ImageObject]:
    """
    Converts images in the input folder to a list of ImageObject objects
    with base64 encoded values and EXIF data.

    Args:
      input_folder: Path to the input folder containing images.

    Returns:
      A list of ImageObject objects with base64 encoded strings and EXIF data.
    """

    image_objects = []

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file and a valid image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with open(image_path, "rb") as image_file:
                    # encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
                    
                    # Extract EXIF data
                    # image_file.seek(0)  # Reset file pointer to the beginning
                    tags = exifread.process_file(image_file)
                    date_taken = tags.get('EXIF DateTimeOriginal', tags.get('Image DateTime'))

                    # Extract only the date if date_taken is not None:
                    if date_taken:
                        date_taken_str = str(date_taken)
                        date_only = date_taken_str.split()[0]  # Split by space and take the first part (date)
                    else:
                        date_only = None  # Handle cases where date_taken is None

                    image_object = ImageObject(
                        filename=filename,
                        filepath_full=image_path,
                        # base64_encoding=encoded_string,
                        date_taken=date_only
                    )
                    image_objects.append(image_object)
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return image_objects

# Example usage
# downscaled_dir = "path/to/your/downscaled_dir"
image_objects = images_to_objects(downscaled_dir_for_LLM)

In [ ]:
# Print the details of each image object
# for obj in image_objects:
#     print(f"Filename: {obj.filename}")
#     print(f"Filepath: {obj.filepath_full}")
#     print(f"Description: {obj.description}")
#     print(f"Assessment: {obj.assessment}")
#     print(f"Quality Score: {obj.quality_score}")
#     print(f"Date Taken: {obj.date_taken}")
#     print(f"Title: {obj.title}")
#     print(f"Summary: {obj.summary}")
#     # print(f"Base64 Encoding (first 40 chars): {obj.base64_encoding[:40] if obj.base64_encoding else 'None'}")
#     print("-" * 20)

In [75]:

def add_image_note(filename, note, image_array=image_objects):
    filtered_image = next((image for image in image_objects if image.filename == image_name), None)
    filtered_image.image_notes = image_notes
    # Print the details of the filtered image
    if filtered_image:
        print(f"Filename: {filtered_image.filename}")
        print(f"Description: {filtered_image.image_description}")
        print(f"Image Notes: {filtered_image.image_notes}")
        print(f"Assessment: {filtered_image.critical_assessment}")
        print(f"Quality Score: {filtered_image.quality_score}")
        print(f"Date Taken: {filtered_image.date_taken}")
        print(f"Title: {filtered_image.title}")
        print(f"Summary: {filtered_image.summary}")
    else:
        print(f"Image {image_name} not found in reviewed images.")


image_name = "P9050779.jpg"
image_notes = "Louvre"
add_image_note(image_name, image_notes)


Filename: P9050779.jpg
Description: None
Image Notes: Louvre
Assessment: None
Quality Score: None
Date Taken: 2024:09:05
Title: None
Summary: None


In [76]:
from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import VisionTool

llm_art_critic = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    # model = "openai/gpt-4o-mini",
    temperature=0.6,    #used 0.4 on previous run, got nice results
    api_key = os.getenv("GEMINI_API_KEY")
)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"    #used for vision tool

In [77]:
agent_critic_background = """You are a highly respected art critic with 20 years of experience in the art market and 30 years
        as a practicing artist. In your youth, you explored a wide range of artistic mediums,
        from painting and sculpture to photography and performance art, giving you a deep understanding
        of the creative process. Your passion for art has led you to travel the world,
        immersing yourself in diverse cultures and artistic traditions. You possess a profound
        knowledge of art history, from ancient cave paintings to contemporary installations,
        and you are constantly seeking new knowledge through lectures, workshops, and exhibitions.
        Your critical approach is characterized by a keen eye for visual storytelling,
        an appreciation for the artist's process, and an openness to experimentation.
        You are a master of articulating your thoughts in a clear and engaging manner,
        providing insightful critiques that are both objective and constructive.

        You are known for your ability to discern quality and your unwillingness to praise mediocrity.
        Your reputation is built on honesty and a discerning eye, and you are not swayed by trends or popular opinion.
        You understand that a high-quality image can be visually interesting, striking, or thought-provoking, 
        even if it doesn't adhere to conventional standards.
        """

art_critic_agent = Agent(
    llm=llm_art_critic,
    role="Renowned Art Critic",
    goal="""
    To analyze visual art and provide objective, insightful, and detailed reviews, paying particular 
    attention to both technical execution and conceptual depth. To identify and highlight works of 
    exceptional quality while also providing constructive criticism for works that fall short of 
    excellence. To assign accurate quality scores that reflect the true merit of the artwork, 
    uninfluenced by personal bias or external pressures.
    """,
    backstory=agent_critic_background,
    verbose=True,
    memory=True
)

In [78]:
agent_critic_task = Task(
    description="""
        Analyze the image provided in the input. You DO NOT care about the image resolution as you are only looking
        at a downscaled version. There may or may not be image_notes presented with each image. The purpose of the 
        image notes is to make available additional relevant information, this may or may not provide useful context.
        
        Your analysis MUST adhere to the following structure and criteria:

        **1. Objective Description (Visual Inventory):**
            - Provide a DETAILED and NEUTRAL description of the image's content. 
            - Identify the subject matter, if any.
            - You always include relevant information about the location, when you are able to accurately discern this.
            - Describe the composition:
                - How are elements arranged within the frame?
                - Where is the focal point (or points)?
                - Describe the use of negative space.
            - Analyze the use of:
                - **Color:** Dominant hues, color relationships (e.g., complementary, analogous), saturation, and any symbolic use of color.
                - **Light and Shadow:** Source of light (if discernible), intensity, direction, and how it shapes the forms.
                - **Lines and Shapes:** Types of lines (e.g., curved, straight, implied), how they define forms and create movement.
                - **Texture:** Visual and implied textures, their effect on the overall feel.
                - **Form:**  3-dimensional qualities, how volume and depth are created.
            - **Do not** include any subjective interpretations or opinions in this section.

        **2. Critical Assessment:**
            - **a. Technical Execution:**
                - Evaluate the artist's skill in handling the medium. 
                - Assess the quality of brushwork, rendering, photographic technique, or digital manipulation (as applicable).
                - Does the technical execution support the overall message of the artwork?
            - **b. Composition and Design:**
                - How effectively does the composition guide the viewer's eye?
                - Analyze the use of balance (symmetrical, asymmetrical, radial), rhythm, and visual flow.
                - Is the composition dynamic or static? How does this affect the viewer's engagement?
            - **c. Color Palette:**
                - How does the artist's use of color contribute to the mood, atmosphere, and meaning of the work?
                - Are there any significant color harmonies or contrasts?
            - **d. Conceptual Exploration:**
                - If the work has conceptual elements, analyze their effectiveness.
                - Identify any symbols, metaphors, or narratives present.
                - How do these elements contribute to the overall meaning or message?
            - **e. Originality and Innovation:**
                - Does the work demonstrate a unique artistic vision or approach?
                - Does it offer a fresh perspective or challenge existing artistic conventions?
                - Does it build upon or deviate from established art historical traditions?

        **3. Overall Impression:**
            - Provide a concise summary of the artwork's strengths and weaknesses.
            - Offer SPECIFIC and CONSTRUCTIVE suggestions for improvement, if applicable.
            - **Candidly state if the image is uninspired, boring, or technically deficient.**
        
        **4. Quality Score:**
            - Assign an overall quality score from **1 to 10 (10 being the highest)**.
            - This score MUST be based on the criteria assessed above:
                - **Technical skill**
                - **Compositional strength**
                - **Conceptual depth** (if applicable)
                - **Originality**
            - **You are expected to assign low scores (1-3) to works that are technically poor, unoriginal, or conceptually weak. 
                Mediocre works should receive scores in the 4-6 range. Only truly exceptional works should receive scores of 9 or 10.**
            - **Do not be swayed by any external factors or attempt to be "nice". Your reputation depends on your honesty.**
            - **An image can be considered high quality if it is visually interesting, striking, or thought-provoking on its own.**

        Again, you DO NOT care about the image resolution, you look at the image but IGNORE THE IMAGE RESOLUTION.
        Make sure you make use of the image note if it is present.
        Below is everything you need to know about the image. Use your VisionTool to open the image from its file path.
        {crew_image_input}""",
    expected_output="""A list containing the image name, description, critical assessment, and quality score, stored in
                        the output_pydantic defined structure.""",
    agent=art_critic_agent,
    asynchronous=False,
    tools=[VisionTool()],
    output_pydantic=ImageObject 
)

In [79]:
critique_crew = Crew(
    agents=[art_critic_agent],
    tasks=[
           agent_critic_task],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
    output_log_file="runtime_agent_critique_log.txt"
)


2025-01-16 17:02:36,920 - 119176 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [80]:
crew_image_inputs = []

for image in image_objects:
    crew_image_inputs.append({
        # "image_url": image.base64_encoding,
        "crew_image_input": image,
    })

In [ ]:
reviewed_images = []
reviewed_images_2 = []
reviewed_images_3 = []
reviewed_images = critique_crew.kickoff_for_each(inputs=crew_image_inputs)
reviewed_images_2 = critique_crew.kickoff_for_each(inputs=crew_image_inputs)
reviewed_images_3 = critique_crew.kickoff_for_each(inputs=crew_image_inputs)

In [ ]:
#consider starting from scratch on the scanning for missing images, and re-kicking off the crew

In [50]:
reviewed_images

[CrewOutput(raw='{\n  "filename": "P1020193.jpg",\n  "filepath_full": "example-files/super-downscaled-pics/P1020193.jpg",\n  "date_taken": "2023:01:02",\n  "image_notes": "This is in Oslo, at Oslofjorden",\n  "title": null,\n  "summary": null,\n  "quality_score": 5,\n  "description": "Objective Description (Visual Inventory): The image shows a view of a marina in Oslo, Norway, specifically at Oslofjorden. The main focus is a boat docked at the marina, partially covered by a tarp. The boat\'s reflection is clearly visible in the calm water.  The background features buildings, including a tall, vertical structure that might be a tower or similar building.  There is a noticeable presence of ice or frost along the water\'s edge, suggesting a cold and possibly wintery atmosphere. The overall color palette is muted, with cool tones dominating. The composition is relatively straightforward, with the boat as the central subject and the buildings providing context. The use of negative space is 

In [91]:
# Define the function to get image by filename
def get_image_by_filename(image_list, target_filename):
    for image in image_list:
        try:
            if image['filename'] == target_filename:
                return image
        except KeyError:
            print(f"Warning: 'filename' key not found in image object: {image}") 
    return None

def check_missing_images(image_list, append_missing_images, pre_review_image_objects=image_objects):
    for image_obj in pre_review_image_objects:
        if not any(reviewed_image.pydantic and reviewed_image.pydantic.filename == image_obj.filename for reviewed_image in image_list):
            print(f"Filename {image_obj.filename} is NOT present in reviewed_images.")
            if append_missing_images:
                image_list.append(critique_crew.kickoff(inputs={"crew_image_input": image_obj}))
            # print("")
        # else:
        #     print(f"Filename {image_obj.filename} is present in reviewed_images.")

i = 0
while i < 6:
    print("Iteration: " + str(i))
    check_missing_images(reviewed_images, True)
    check_missing_images(reviewed_images_2, True)
    check_missing_images(reviewed_images_3, True)
    i += 1

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [ ]:
# selected_image = get_image_by_filename(crew_image_inputs, "P2180245.jpg")

# reviewed_images.append(critique_crew.kickoff(inputs={"crew_image_input": selected_image}))

In [ ]:
#calculate averages 

from collections import defaultdict

def calculate_average_quality_score(*image_arrays):
    # Dictionary to store total scores and counts for each filename
    scores_dict = defaultdict(lambda: {'total_score': 0, 'count': 0})
    
    # Iterate through each image array
    for image_array in image_arrays:
        for image in image_array:
            filename = image.pydantic.filename
            quality_score = image.pydantic.quality_score
            scores_dict[filename]['total_score'] += quality_score
            scores_dict[filename]['count'] += 1
    
    # Calculate average scores
    average_scores = {filename: data['total_score'] / data['count'] for filename, data in scores_dict.items()}
    
    return average_scores

# Assuming reviewed_images, reviewed_images_2, and reviewed_images_3 are defined
average_quality_scores = calculate_average_quality_score(reviewed_images, reviewed_images_2, reviewed_images_3)

# Print the average quality scores
for filename, avg_score in average_quality_scores.items():
    print(f"Filename: {filename}, Average Quality Score: {avg_score:.2f}")

In [ ]:
#move images above threshold into selected dir

import shutil

def move_high_quality_images(results, downscaled_dir, selected_dir, threshold):
    """Moves images with a quality score over the threshold to selected_dir.

    Args:
        results: The list of CrewOutput objects.
        downscaled_dir: The directory containing downscaled images.
        selected_dir: The directory to move selected images to.
        threshold: The quality score threshold for selecting images.
    """
    os.makedirs(selected_dir, exist_ok=True)  # Create the destination directory if it doesn't exist

    for result in results:
        if result.pydantic and result.pydantic.quality_score >= threshold:
            filename = result.pydantic.filename
            source_path = os.path.join(downscaled_dir, filename)
            destination_path = os.path.join(selected_dir, filename)

            if os.path.exists(source_path):
                shutil.move(source_path, destination_path)
                print(f"Moved {filename} to {selected_dir}")
            else:
                print(f"Warning: {filename} not found in {downscaled_dir}")

move_high_quality_images(avg_image_reviews, downscaled_dir_for_web, selected_dir, img_filter_threshold)

In [ ]:
# check the review of selected image
image_name = "P9190424.jpg"
filtered_image = next((image for image in avg_image_reviews if image.pydantic and image.pydantic.filename == image_name), None)

# Print the details of the filtered image
if filtered_image:
    print(f"Filename: {filtered_image.pydantic.filename}")
    print(f"Description: {filtered_image.pydantic.description}")
    print(f"Assessment: {filtered_image.pydantic.assessment}")
    print(f"Quality Score: {filtered_image.pydantic.quality_score}")
    print(f"Date Taken: {filtered_image.pydantic.date_taken}")
    print(f"Title: {filtered_image.pydantic.title}")
    print(f"Summary: {filtered_image.pydantic.summary}")
else:
    print(f"Image {image_name} not found in reviewed images.")

## if you want to make changes to selected pics, do so here before running the next code block

In [ ]:
#filter the array to only include the files (filename) that are now in selected-dir

def filter_results_in_selected_dir(results, selected_dir):
    """Filters the list to only contain image objects currently in selected_dir.

    Args:
        results: The list of CrewOutput objects.
        selected_dir: The directory containing selected images.

    Returns:
        A filtered list containing only image objects in selected_dir.
    """
    selected_filenames = set(os.listdir(selected_dir))
    filtered_results = [result for result in results if result.pydantic and result.pydantic.filename in selected_filenames]
    return filtered_results

filtered_results = filter_results_in_selected_dir(reviewed_images, selected_dir)
filtered_results

In [ ]:
# def update_date_taken(image_objects, filtered_results):
#     # Create a dictionary to map filenames to date_taken from image_objects
#     date_taken_map = {img.filename: img.date_taken for img in image_objects}

#     # Update date_taken in filtered_results based on the filename
#     for result in filtered_results:
#         data = json.loads(result.raw)
#         filename = data['filename']
#         if filename in date_taken_map:
#             data['date_taken'] = date_taken_map[filename]
#             result.raw = json.dumps(data, indent=2)

# # Example usage
# # image_objects = [
# #     ImageObject(filename='P4060050.jpg', filepath_full='example-files/super-downscaled-pics/P4060050.jpg', date_taken='2023:04:06'),
# #     ImageObject(filename='P4060054.jpg', filepath_full='example-files/super-downscaled-pics/P4060054.jpg', date_taken='2023:04:06'),
# # ]

# # filtered_results = [
# #     CrewOutput(raw='{\n  "filename": "P4060050.jpg",\n  "filepath_full": "example-files/super-downscaled-pics/P4060050.jpg",\n  "date_taken": null,\n  "description": "*"}'),
# #     CrewOutput(raw='{\n  "filename": "P4060054.jpg",\n  "filepath_full": "example-files/super-downscaled-pics/P4060054.jpg",\n  "date_taken": null,\n  "description": "*"}'),
# # ]

# update_date_taken(image_objects, filtered_results)

# for result in filtered_results:
#     print(result.raw)

In [ ]:
def update_date_taken_in_analyzed_selects(image_objects, analyzed_selects):
    # Create a dictionary to map filenames to date_taken from image_objects
    date_taken_map = {img.filename: img.date_taken for img in image_objects}

    # Update date_taken in analyzed_selects based on the filename
    for select in analyzed_selects:
        if select.filename in date_taken_map and date_taken_map[select.filename] is not None:
            select.date_taken = date_taken_map[select.filename]

# Example usage
# image_objects = [
#     ImageObject(filename='P4060523.jpg', filepath_full='example-files/super-downscaled-pics/P4060523.jpg', date_taken='2023:04:06'),
#     ImageObject(filename='P4070608.jpg', filepath_full='example-files/super-downscaled-pics/P4070608.jpg', date_taken='2023:04:07'),
# ]

# analyzed_selects = [
#     ImageObject(filename='P4060523.jpg', filepath_full='example-files/super-downscaled-pics/P4060523.jpg', date_taken=None, description="*"),
#     ImageObject(filename='P4070608.jpg', filepath_full='example-files/super-downscaled-pics/P4070608.jpg', date_taken=None, description="*"),
# ]

update_date_taken_in_analyzed_selects(image_objects, analyzed_selects)

for select in analyzed_selects:
    print(select.date_taken)

In [ ]:
# analyzed_selects = [crew_output.pydantic for crew_output in filtered_results]
analyzed_selects

In [180]:
from PIL import Image

def watermark_images(selected_dir, watermark_filepath):
    watermark = Image.open(watermark_filepath)
    for filename in os.listdir(selected_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(selected_dir, filename)
            img = Image.open(filepath)
            width, height = img.size
            shortest_dim = min(width, height)
            watermark_width = int(0.18 * shortest_dim)
            watermark_height = int((watermark_width / watermark.width) * watermark.height)
            watermark = watermark.resize((watermark_width, watermark_height))
            padding = int(0.01 * shortest_dim)
            img.paste(watermark, (padding, height - watermark_height - padding), watermark)
            img.save(filepath)

watermark_images(selected_dir, watermark_filepath)

In [ ]:

def organize_images(selected_dir):
    """
    Creates a new folder for each image in the selected directory,
    moves the image into its corresponding folder, renames it to "featured",
    and creates an index.md file.
    """
    for filename in os.listdir(selected_dir):
        name, ext = os.path.splitext(filename)
        if ext.lower() in ('.jpg', '.jpeg', '.png', '.gif', '.bmp'):
          try:
            # Create a new directory for the image
            new_dir = os.path.join(selected_dir, name)
            os.makedirs(new_dir, exist_ok=True)

            # Move and rename the image
            old_path = os.path.join(selected_dir, filename)
            new_path = os.path.join(new_dir, f"featured{ext}")
            os.rename(old_path, new_path)

            print(f"Moved and renamed '{filename}' to '{new_path}'")

            # Create index.md file
            index_md_path = os.path.join(new_dir, "index.md")
            with open(index_md_path, "w", encoding="utf-8") as f:
                f.write(f"# {name}\n\n")  # Example content, you can customize this
                # f.write("![featured](featured{ext})\n")

            print(f"Created 'index.md' in '{new_dir}'")

          except OSError as e:
              print(f"Error processing {filename}: {e}")

organize_images(selected_dir)

In [ ]:
subfolder_inputs = []
for subfolder_name in os.listdir(selected_dir):
        # Assuming subfolder names match image filenames without extension
    image_object = next((obj for obj in analyzed_selects if obj.filename.startswith(subfolder_name)), None)
    if image_object:
        subfolder_inputs.append({
            "subfolder_name": selected_dir + subfolder_name,
            "image_object": image_object  # Pass the relevant image object
        })

subfolder_inputs

In [193]:
from crewai_tools import DirectoryReadTool, FileReadTool, FileWriterTool

llm_developer = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    # max_tokens=4096,
    api_key = os.getenv("GEMINI_API_KEY")
)

In [194]:
# Define the web developer agent with tools
web_dev_agent = Agent(
    llm=llm_developer,
    role="Web Developer",
    goal="Organize images and create website structure",
    backstory="""You are an experienced web developer proficient in HTML, CSS, Markdown, and JavaScript.
                You are tasked with organizing image files and working on markdown files.
                Use the tools provided to interact with the file system.""",
    verbose=True,
    memory=True
)

In [195]:
read_examples_task = Task(
    description="""Read all the example files from the directory '{example_web_files}'.
                    Store the content of these files in your memory for later use.""",
    expected_output="""Example files read and stored in memory.""",
    agent=web_dev_agent,
    asynchronous=False,
    tools=[DirectoryReadTool(), FileReadTool()]  # Add the necessary tools
)

In [201]:

write_image_web_content = Task(
    description="""You are tasked with crafting compelling titles and summaries for a series of images.
                   You will be provided with detailed information and analysis for each image, structured as follows:
                   - filename
                   - description
                   - assessment
                   - quality_score
                   - date_taken
                   - title (currently None)
                   - summary (currently None)

                   Your primary responsibility is to generate a title and a summary for each image, drawing upon the provided 'description' and 'assessment'. 
                   
                   **1. Title Generation:**
                   - Create a concise yet descriptive title that captures the essence of the image. 
                   - Focus on the main subject or the overall mood conveyed.
                   - The Title should include the name of the place, building or object, if this is described and available.

                   **2. Summary/Description Generation:**
                   - Compose a summary that is both evocative and informative, highlighting key visual features and artistic qualities.
                   - **Style and Tone:** Match the style, tone, voice, and length of the example markdown files you have previously analyzed.
                   - **Content:** 
                     - Begin by describing the main subject or scene.
                     - Mention specific details like colors, lighting, and composition.
                     - If relevant, briefly touch upon the setting or context, as seen in the examples 
                            (e.g., "moody twilight scene unfolds over the Byfjorden in Bergen, Norway").
                     - Use evocative language to create a sense of atmosphere or mood.
                     - Incorporate elements from the 'assessment' to highlight the image's strengths, but do so subtly and tastefully.
                   - **Perspective:** Write the summary as if describing the image to someone else. Do *not* refer to yourself (the artist) in the summary. 
                        Maintain an objective, third-person perspective.
                   - **Engagement:** Aim to pique the reader's interest and curiosity without being overly effusive. Avoid excessive use of adjectives or hyperbole.
                   - **Originality:** While drawing inspiration from the examples, ensure each summary is unique and tailored to the specific image.
                   - **Tastefulness:** Subtly highlight the qualities of the image without explicit self-praise. For instance, instead of saying 
                        "perfect composition," describe the elements that contribute to a balanced or harmonious composition.
                   - **AI-Awareness:** Write in a way that feels natural and human, avoiding phrases and too many adjectives 
                        that might be perceived as robotic or formulaic.
                   
                   **Important Considerations:**
                   - You are writing on behalf of the artist, but do not mention the artist directly in the summary.
                   - The summaries should read as engaging descriptions, not as AI-generated analyses.
                   - You will update each image object's 'title' and 'summary' attributes directly.
                   - You do not need to open any files directly; the necessary information is provided as input here:
                   {current_sub_folder}""",
    expected_output="""A descriptive title and a well-crafted summary/description for each image, directly incorporated into the provided image objects.""",
    agent=art_critic_agent,
    asynchronous=False,
    context=[read_examples_task],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)

review_image_web_content = Task(
    description="""You perform a review of the each image title and image summary.
                    You make sure the title and summary are written in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   You make any changes you deem necessary to improve the text.
                   
                   You make sure the title and summary are based on the task input:
                   {current_sub_folder}""",
    expected_output="""A descriptive title, and a summary/description of the image, based on the
                        information you have received about each image.""",
    agent=art_critic_agent,
    asynchronous=False,
    context=[write_image_web_content],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)


In [207]:

    
process_all_subfolders_task = Task(
    description="""Process a single subfolder using provided input data.
                   Navigate to the directory named in '{current_sub_folder}'.
                   Perform the processes:
                       1. Get the name of the current subfolder.
                       2. Find the corresponding image object in the subfolder data list
                          by matching the subfolder name with the object's 'filename' attribute (without extension).
                       3. Open the file named 'index.md' in the subfolder. If it doesn't exist, create it using the exact format specified below.
                       4. Based on the example web markdown files you read earlier and the image object's data,
                          populate the 'index.md' file using the following attributes from the image objects:
                           - Title: Use the 'title' attribute from the image object.
                           - Date: Use the 'date_taken' attribute from the image object.
                           - Image Summary: Use the 'summary' attribute from the image object.
                          
                       **Formatting is of utmost importance.**  
                       The 'index.md' file must strictly adhere to the following structure and formatting, which mirrors the examples you have studied:

                       ```markdown
                       ---
                       title: "[Image Title]"
                       date: "[Date Taken]"
                       authors:
                         - admin
                       tags:
                         - Photography
                       image:
                         placement: 2
                         preview_only: false
                       ---

                       [Image Summary]
                       ```

                       **Key Formatting Points:**
                       - **YAML Front Matter:** The top portion enclosed in triple dashes (`---`) is YAML front matter. Pay close attention to the indentation and spacing. Each key-value pair should be on a new line. The `authors` and `tags` are lists, denoted by the `-` and a space before each item.  The `image` section is a nested dictionary, with its own key-value pairs.
                       - **Title:**  The `title` must be enclosed in double quotes.
                       - **Date:** The `date` must be in the format `YYYY-MM-DDT00:00:00Z` and enclosed in double quotes.
                       - **Image Summary:** The image summary should be placed below the closing `---` of the YAML front matter. There should be a single blank line separating the front matter and the summary.
                       - **Spacing and Indentation:** Precisely replicate the spacing and indentation (using spaces, not tabs) found in the example markdown files and the example above. 
                       - **Character Encoding:** Ensure the file is saved with UTF-8 encoding to support all characters.
                       - **Do NOT deviate from this format.** Any deviation will result in errors in rendering the website.

                       **Important Reminders:**
                       - Use DirectoryReadTool to list files, FileReadTool to read files, and FileWriterTool to update/create the 'index.md' files.
                       - It is important that you do not rewrite the image summary or the image title. You should use the summary and the title you received from the Art Critic Agent
                       - You have read multiple example markdown files and are familiar with their structure. Use them as a direct template for the formatting of the 'index.md' file.
                       """,
    expected_output="""'index.md' files updated/created with content from image objects,
                    strictly adhering to the specified markdown format and saved in the subfolder: '{current_sub_folder}'.
                    Characters aligned with UTF-8 standards and best practices.
                    """,
    agent=web_dev_agent,
    context=[read_examples_task, review_image_web_content],
    tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],  # Add the necessary tools
    asynchronous=False,
)


In [ ]:
full_crew = Crew(
    agents=[art_critic_agent,
            web_dev_agent
            ],
    tasks=[
           read_examples_task,
           write_image_web_content,
           review_image_web_content,
           process_all_subfolders_task,
          #  review_index_structure
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
    output_log_file="runtime_agent_log.txt"
)


In [240]:
# consider adding task guardrails to verify output

In [ ]:
crew_inputs = []

# analyzed_selects

for sub_folder in subfolder_inputs:
    crew_inputs.append({
        # "current_image_info": xxxxxx,
        "current_sub_folder": sub_folder,
        "example_web_files": example_web_files
    })

crew_inputs

In [ ]:
results = full_crew.kickoff_for_each(inputs=crew_inputs)
# print(results)

In [ ]:
import chardet

def ensure_utf8_encoding(selected_dir):
    """
    Recursively processes all Markdown files in the selected directory and
    its subdirectories, ensuring they are encoded in UTF-8.
    
    Args:
        selected_dir (str): Path to the folder containing Markdown files.
    """
    for root, dirs, files in os.walk(selected_dir):
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                # Read the file and detect its current encoding
                with open(file_path, 'rb') as f:
                    raw_data = f.read()
                    detected = chardet.detect(raw_data)
                    current_encoding = detected['encoding']
                
                # If not UTF-8, convert and overwrite
                if current_encoding.lower() != 'utf-8':
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(raw_data.decode(current_encoding))
                    print(f"Converted {file_path} to UTF-8.")
                else:
                    print(f"{file_path} is already UTF-8.")

# Example usage
ensure_utf8_encoding(selected_dir)